# Downloading Your Azure ML Model for Local Use

This notebook shows how to download your registered Azure ML model for local inference. This step is essential for situations where you need to run your model locally or in environments that don't have direct access to Azure.

## What You'll Learn

- How to authenticate with Azure Machine Learning
- How to connect to your Azure ML workspace
- How to download a registered model
- How to structure your local files for inference

## Prerequisites

- Completed the previous notebooks:
  - `01.AzureML_Distillation.ipynb` (generated training data)
  - `02.AzureML_FineTuningAndConvertByMSOlive.ipynb` (fine-tuned and optimized the model)
  - `03.AzureML_RuningByORTGenAI.ipynb` (tested the optimized model)
  - `04.AzureML_RegisterToAzureML.ipynb` (registered your model to Azure ML)
- A model registered in your Azure ML workspace
- Python environment with necessary libraries (which we'll install)

## Setup Instructions

1. **Azure Authentication**: Ensure you're logged in to Azure using `az login --use-device-code` in a terminal
2. **Kernel Selection**: Change the Jupyter kernel to "Python 3.10" using the selector in the top right
3. **Environment File**: Ensure your `local.env` file exists with proper Azure ML workspace information

## System Requirements

**Python Version**: This notebook requires Python 3.10 or higher installed on your local machine. 
If you're running this locally on your own machine or VM, ensure you have Python 3.10+ installed before proceeding.

## Local Execution Note

This notebook is designed for local execution, as its purpose is to download the model from Azure ML to your local environment. If you're viewing this in Azure ML Studio, you'll need to:

1. Download this notebook to your local machine
2. Install the required libraries locally
3. Configure your environment with the proper Azure credentials

The notebook will download potentially large model files (hundreds of MB), so ensure you have sufficient disk space.

## Azure Authentication

Before running this notebook, you must authenticate with Azure. Open a terminal or command prompt and run:

```bash
az login --use-device-code
```

This will provide a code and a URL. Go to the URL, enter the code, and complete the authentication process. This login will be used by the Azure SDK to access your Azure ML resources.

Alternatively, if you have Azure CLI installed, you can authenticate using the credentials stored there.

## 1. Package Installation Helper Function

First, we define a helper function to manage package installation. This function:

1. Checks if a specified package is already installed by attempting to import it
2. If the package is installed, confirms with a checkmark message
3. If the package is not installed, uses pip to install it

This approach ensures we have all the necessary packages without reinstalling ones that are already present, making the notebook more efficient and avoiding potential version conflicts.

## 2. Install and Verify Required Packages

Now we use our helper function to install all the packages needed for this notebook:

1. **Core Dependencies**:
   - `python-dotenv`: For loading environment variables
   - `azure-identity`: For Azure authentication
   - `azure-ai-ml`: The Azure ML SDK for accessing models
   - `tqdm`: For progress bars during downloads
   - `ipywidgets`: For interactive Jupyter widgets

2. We explicitly update `azure-ai-ml` to ensure we have the latest version, as this package is frequently updated with new features

3. **Verification**:
   - Import key Azure modules to confirm they're installed correctly
   - Test if Azure authentication is available through `DefaultAzureCredential`
   - Provide a warning if authentication isn't configured correctly

This comprehensive setup ensures we have everything needed for downloading the model from Azure ML.

In [ ]:
# Install all required packages
import sys
import subprocess
import importlib

def install_package(package_name):
    try:
        __import__(package_name)
        print(f"✓ {package_name} is already installed")
    except ImportError:
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"✓ {package_name} installed successfully")

# Install essential packages
install_package('python-dotenv')
install_package('azure-identity')
install_package('azure-ai-ml')
install_package('tqdm')
# Install ipywidgets for Jupyter progress bars
install_package('ipywidgets')

# For safety, ensure azure-ai-ml is latest version
print("\nUpdating azure-ai-ml to latest version...")
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'azure-ai-ml', '--upgrade'])
print("✓ azure-ai-ml updated to latest version")

# Verify the Azure ML SDK installation
try:
    from azure.ai.ml import MLClient
    from azure.identity import DefaultAzureCredential
    
    # Check if az CLI is installed and user is logged in
    try:
        # This will fail if az CLI isn't installed or user isn't logged in
        credential = DefaultAzureCredential()
        print("✓ Azure authentication is available (DefaultAzureCredential)")
    except Exception as e:
        print(f"⚠️ Azure authentication issue: {str(e)}")
        print("Please run 'az login' in your terminal to authenticate")
        
except Exception as e:
    print(f"⚠️ Issue with Azure ML SDK: {str(e)}")
    print("Please ensure your Azure ML SDK installation is correct")

## Library Update Check

Let's also confirm that we have the latest version of the Azure ML SDK. This explicit update ensures we have all the latest features and bug fixes for working with Azure ML.

## 3. Ensure Latest Azure ML SDK

This command makes sure we have the very latest version of the Azure ML SDK installed. The `-U` flag tells pip to upgrade the package if it's already installed.

Having the latest version is important because:
- New features are frequently added
- Bug fixes are applied regularly
- Compatibility with Azure services is maintained

This is especially important for the download functionality, which may have been enhanced in newer versions.

In [ ]:
! pip install azure-ai-ml -U  

## Azure Authentication

Before using this notebook, make sure you're authenticated with Azure by running:

```
az login --use-device-code
```

This will authenticate your session and allow the notebook to connect to your Azure resources.

## 4. Import Required Packages

Here we import all the packages needed for downloading the Azure ML model:

- **os**: For file and path operations
- **time**: For adding delays or timing operations
- **sys**: For system-level operations
- **azure.ai.ml**: The Azure ML SDK for accessing models and workspaces
- **azure.identity**: For Azure authentication
- **dotenv**: For loading environment variables
- **tqdm**: For displaying progress bars during download

After importing, we print a confirmation message to ensure all packages were imported successfully without errors.

In [ ]:
# Import required packages
import os
import time
import sys
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

# Import regular tqdm instead of notebook-specific version
from tqdm import tqdm

print("All packages successfully imported")

In [ ]:
import os
from dotenv_azd import load_azd_env
from dotenv import load_dotenv

# Load environment variables from current AZD environment if available
load_azd_env(quiet=True)

# Load environment variables from local.env file if it exists
load_dotenv(dotenv_path="local.env")

## 5. Load Environment Variables

This cell loads the environment variables that contain our Azure ML workspace configuration:

1. First, we try to load from an Azure Developer CLI (AZD) environment if available using `load_azd_env()`

2. Then, we load from a local `.env` file using `load_dotenv()`. This serves as a backup if AZD environment variables aren't present.

The environment variables we need are:
- AZUREML_SUBSCRIPTION_ID: Your Azure subscription ID
- AZUREML_RESOURCE_GROUP: The resource group containing your Azure ML workspace
- AZUREML_WS_NAME: The name of your Azure ML workspace

These will be used to connect to your workspace and access your model.

In [ ]:
# Get Azure ML credentials from environment variables
subscription_id = os.getenv('AZUREML_SUBSCRIPTION_ID')
resource_group = os.getenv('AZUREML_RESOURCE_GROUP')
workspace = os.getenv('AZUREML_WS_NAME')

# Check if any variable is missing
missing_vars = []
if not subscription_id:
    missing_vars.append("AZUREML_SUBSCRIPTION_ID")
if not resource_group:
    missing_vars.append("AZUREML_RESOURCE_GROUP")
if not workspace:
    missing_vars.append("AZUREML_WS_NAME")

if missing_vars:
    print(f"❌ Error: The following required environment variables are missing: {', '.join(missing_vars)}")
    print("Please verify that these variables are set in your current azd environment.")
else:
    # Print values for debugging
    print(f"✓ Subscription ID: {subscription_id}")
    print(f"✓ Resource Group: {resource_group}")
    print(f"✓ Workspace: {workspace}")
    
    try:
        # Create ML client with the credentials
        print("Connecting to Azure ML...")
        ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)
        print("✓ Successfully connected to Azure ML workspace")
    except Exception as e:
        print(f"❌ Error connecting to Azure ML: {str(e)}")
        print("Please check your credentials and make sure you're logged in (az login)")

## 6. Retrieve Azure ML Workspace Information

Now we retrieve the Azure ML workspace information from our environment variables:

1. **subscription_id**: The Azure subscription that contains your workspace

2. **resource_group**: The Azure resource group that contains your workspace

3. **workspace**: The name of your Azure ML workspace

These values will be used in the next step to create a connection to your Azure ML workspace. If any of these values are not properly set in your environment variables, you'll need to check your `.env` file or set them directly.

## 7. List Available Models

This cell connects to your Azure ML workspace and lists all available models. This helps you:

1. Verify that your connection to Azure ML is working properly

2. See all models available in your workspace, along with their versions

3. Identify the specific model you want to download

If no models are found, it could indicate:
- The workspace is empty
- You don't have proper permissions
- The workspace information is incorrect

The code includes comprehensive error handling to help diagnose any connection issues.

In [ ]:
# List available models in the workspace
try:
    print("Listing available models in workspace...")
    models = list(ml_client.models.list())
    
    if models:
        print(f"Found {len(models)} models in the workspace:")
        for model in models:
            print(f" - {model.name} (version: {model.version})")
    else:
        print("No models found in the workspace.")
        print("Please verify that you have the correct workspace and permissions.")
        
except Exception as e:
    print(f"❌ Error listing models: {str(e)}")
    print("This could be due to incorrect credentials or Azure ML workspace information.")
    print("Please verify that the environment variables in your azd environment are correct.")

## Download the Fine-Tuned Model

Now we'll download the fine-tuned Phi-4-Mini model from Azure ML Model Registry. This process will:

1. Connect to Azure ML using our credentials
2. Download the ONNX model (int4 quantized for CPU)
3. Display a progress bar during download
4. Show the download location and model size when complete

After downloading, you'll be able to use this model locally for inference.

## 9. Download and Verify the Model

This cell handles the actual download of our model from Azure ML. The code includes several robust features:

1. **Multiple Download Methods**:
   - Tries a standard synchronous download first
   - Falls back to asynchronous download if the standard method fails
   - Shows download progress indicators during the process

2. **Download Verification**:
   - Checks if the download path exists
   - Calculates the total model size and file count
   - Formats the size to be human-readable (bytes, KB, MB, GB)

3. **Path Handling**:
   - Handles cases where the API doesn't return a path
   - Checks common default locations if the reported path is missing

4. **Comprehensive Error Handling**:
   - Provides clear error messages for different failure scenarios
   - Suggests solutions for common issues

This approach ensures a reliable download process, even in different environments or with large models.

In [ ]:
# Define model name and version
model_name = "fine-tuning-phi-4-mini-onnx-int4-cpu"
model_version = 1

try:
    print(f"\nStarting download of model: {model_name} (version {model_version})\n")
    
    # Simple text-based progress indicator that works in all environments
    print("Downloading model... This may take a few minutes.")
    print("[" + "-" * 50 + "]\r", end="")
    
    # Download the model (using standard download method to avoid progress bar issues)
    try:
        # First try non-async version which works in more environments
        download_start_time = time.time()
        download_path = ml_client.models.download(name=model_name, version=model_version)
        download_duration = time.time() - download_start_time
        
        # Print completed progress bar
        print("[" + "#" * 50 + "]")
        print(f"\nModel downloaded successfully in {download_duration:.1f} seconds")
        print(f"Download path: {download_path if download_path else 'Path not returned by Azure ML API'}")
        
    except Exception as e:
        # If the first method fails, try the async version
        print(f"\nStandard download failed, trying async download: {str(e)}")
        download_start_time = time.time()
        download_future = ml_client.models.download_async(name=model_name, version=model_version)
        
        # Print progress dots while waiting
        dots = 0
        while not download_future.done():
            dots = (dots % 10) + 1
            print(f"Downloading{'.'.ljust(dots, '.')} \r", end="")
            time.sleep(1)
        
        # Get the result
        download_path = download_future.result()
        download_duration = time.time() - download_start_time
        print(f"\nModel downloaded successfully in {download_duration:.1f} seconds")
        print(f"Download path: {download_path if download_path else 'Path not returned by Azure ML API'}")
    
    # Check if download_path is valid before attempting to use it
    if download_path and os.path.exists(download_path):
        total_size = 0
        file_count = 0
        for path, dirs, files in os.walk(download_path):
            for f in files:
                fp = os.path.join(path, f)
                file_size = os.path.getsize(fp)
                total_size += file_size
                file_count += 1
    
        # Convert to appropriate size format (bytes, KB, MB, GB)
        def format_size(size_bytes):
            if size_bytes < 1024:
                return f"{size_bytes} bytes"
            elif size_bytes < 1024**2:
                return f"{size_bytes/1024:.2f} KB"
            elif size_bytes < 1024**3:
                return f"{size_bytes/(1024**2):.2f} MB"
            else:
                return f"{size_bytes/(1024**3):.2f} GB"
    
        print(f"Model details:")
        print(f" - Total size: {format_size(total_size)}")
        print(f" - Files: {file_count}")
        print(f" - Location: {download_path}")
    else:
        # The download seems to have happened but the path is not valid
        print("\nModel download completed, but the path was not returned by the API or is invalid.")
        print("This is sometimes expected behavior when models are downloaded to the default location.")
        
        # Try to find the model in a common default location
        default_path = os.path.join(os.getcwd(), model_name)
        if os.path.exists(default_path):
            print(f"Found model in the default path: {default_path}")
            
            # Calculate model size
            total_size = 0
            file_count = 0
            for path, dirs, files in os.walk(default_path):
                for f in files:
                    fp = os.path.join(path, f)
                    file_size = os.path.getsize(fp)
                    total_size += file_size
                    file_count += 1
            
            print(f"Model details:")
            print(f" - Total size: {format_size(total_size)}")
            print(f" - Files: {file_count}")
            print(f" - Location: {default_path}")
        else:
            print("\nLooking for the model in current directory...")
            model_dirs = [d for d in os.listdir() if os.path.isdir(d) and 
                          (model_name.lower() in d.lower() or "model" in d.lower())]
            
            if model_dirs:
                print(f"Found potential model directories: {', '.join(model_dirs)}")
                print("Please check these directories for your downloaded model.")
            else:
                print("Could not locate the downloaded model in the current directory.")
                print("Please check your Azure ML workspace for the downloaded model location.")
        
except Exception as e:
    print(f"\n❌ Error in model download process: {str(e)}")
    print("Please check your Azure ML permissions and make sure the model exists in your workspace.")
    print("You can list available models with ml_client.models.list() to verify the model name and version.")
    
    # Offer troubleshooting advice based on the error
    error_str = str(e).lower()
    if "progress" in error_str or "iprogress" in error_str:
        print("\nTroubleshooting: The error is related to progress bar display in your environment.")
        print("This has been addressed by using a simpler progress display method.")
    elif "not found" in error_str:
        print("\nTroubleshooting: The model might not exist in your workspace.")
        print("Check the model name and version or run the cell to list models first.")
    elif "credentials" in error_str or "authentication" in error_str:
        print("\nTroubleshooting: You might need to re-authenticate with Azure.")
        print("Run 'az login --use-device-code' in your terminal.")
    elif "subscription" in error_str:
        print("\nTroubleshooting: Your Azure subscription settings might be incorrect.")
        print("Check that your azd environment has the correct AZUREML_SUBSCRIPTION_ID.")
    elif "nonetype" in error_str or "none" in error_str:
        print("\nTroubleshooting: The model download completed, but the API didn't return a path.")
        print("Check the current directory for a folder named after the model.")